In [46]:
import numpy 
import math 
from scipy import integrate
from matplotlib import pyplot 

# 1. Flow around a unit cylider and plot the Pressure coefficient.

In [47]:
# defines the cylinder
R = 1.0                                                           # radius
theta = numpy.linspace(0, 2*math.pi, 100)                         # angles in radians
x_cylinder, y_cylinder = R*numpy.cos(theta), R*numpy.sin(theta)   # coordinates of the cylinder


%matplotlib inline

In [48]:
class Panel:
    """Contains information related to a panel."""
    def __init__(self, xa, ya, xb, yb):
        """Initializes the panel.
        
        Arguments
        ---------
        xa, ya -- coordinates of the first end-point of the panel.
        xb, yb -- coordinates of the second end-point of the panel.
        """
        self.xa, self.ya = xa, ya
        self.xb, self.yb = xb, yb
        
        self.xc, self.yc = (xa+xb)/2, (ya+yb)/2       # control-point (center-point)
        self.length = math.sqrt((xb-xa)**2+(yb-ya)**2)     # length of the panel
        
        # orientation of the panel (angle between x-axis and panel's normal)
        if xb-xa <= 0.:
            self.beta = math.acos((yb-ya)/self.length)
        elif xb-xa > 0.:
            self.beta = math.pi + math.acos(-(yb-ya)/self.length)
            
        self.sigma = 1                             # source strength
        
        self.n = [ math.cos(self.beta), math.sin(self.beta)]
        self.t = [-math.sin(self.beta), math.cos(self.beta)]
        
        
        self.vt = 0                                 # tangential velocity
        self.cp = 0                                 # pressure coefficient

In [49]:
def get_source_panel_velocity(p,x,y):
    """Evaluates the velocity components induced at coordinates (x,y) by a panel p.
    
    Arguments
    ---------
    p -- panel from which the velocity is induced
    x -- x-coordinate in global frame
    y -- y-coordinate in global frame
    
    Returns
    -------
    (u,v)-components of velocity at (x,y)
    """
    
    x_p =  math.sin(p.beta)*(x-p.xb) - math.cos(p.beta)*(y-p.yb)
    y_p =  math.cos(p.beta)*(x-p.xb) + math.sin(p.beta)*(y-p.yb)
    
    u_p = p.sigma/(4*math.pi)*((x_p)/((x_p**2+y_p**2)-(y_p)/(x_p-p.length)**2+y_p**2))
    v_p = p.sigma/(2*math.pi)*((x_p)/((x_p**2+y_p**2) - (x_p-p.length)**2/((x_p-p.length)**2+(y_p**2))
    
    u =  math.sin(p.beta)*u_p + math.cos(p.beta)*v_p
    v = -math.cos(p.beta)*u_p + math.sin(p.beta)*v_p
    
    return u, v

SyntaxError: invalid syntax (<ipython-input-49-34e41e271f81>, line 21)

In [ ]:
class Freestream:
    """Freestream conditions"""
    def __init__(self, u_inf=1.0, alpha=2.0):
        """Sets the freestream conditions
        
        Params:
        -------
        u_inf    float, far field speed defaults to 1.0
        alpha    float, angle of attack in degrees defaults to 0.0
        """
        
        self.u_inf = u_inf
        self.alpha = alpha = alpha*numpy.pi/180

In [ ]:
def get_pressure_coefficient(panels, freestream):
    """Computes the surface pressure coefficients.
    
    Arguments
    ---------
    panels -- array of panels.
    freestream -- farfield conditions.
    """
    for panel in panels:
        panel.cp = 1.0 - (panel.vt/u_inf)**2

In [ ]:
u_inf, alpha = 1.0, 2.0
freestream = Freestream(u_inf, alpha)

In [ ]:
N_panels = 38                   # number of panels desired

# defining the end-points of the panels
x_ends = R*numpy.cos(numpy.linspace(0, 2*math.pi, N_panels+1))
y_ends = R*numpy.sin(numpy.linspace(0, 2*math.pi, N_panels+1))

# defining the panels
panels = numpy.empty(N_panels, dtype=object)
for i in range(N_panels):
    panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])

In [ ]:
A = numpy.empty((N_panels, N_panels), dtype=float)
#numpy.fill_diagonal(A, 0.5)
    
for i in range(N_panels):
    for j in range(N_panels):
        if i != j:
            A[i,j] = numpy.dot(get_source_panel_velocity(panels[j],panels[i].xc,panels[i].yc),panels[i].n)
        else:
            A[i,j] = 0.5*1.001
            
b = numpy.empty((N_panels), dtype='float')
for i in range(N_panels):
    b[i] = (-freestream.u_inf)(numpy.dot(numpy.array([u_inf,0]), (panels[i].n)))

In [ ]:
sigma = numpy.linalg.solve(A, b)

In [ ]:
C = numpy.empty((N_panels, N_panels), dtype='float')
    #numpy.fill_diagonal(A, 0.0)
    
for i in range(N_panels):
    for j in range(N_panels):
        if i != j:
            C[i,j] = numpy.dot(get_source_panel_velocity(panels[j],panels[i].xc,panels[i].yc),panels[i].t)
        else:
            C[i,j] = 0.0

In [ ]:
D = numpy.empty((N_panels), dtype='float')
for i in range(N_panels):
    D[i] = freestream.u_inf.dot(numpy.array([u_inf,0]), (panels[i].t))

In [ ]:
vt = numpy.dot(C,sigma) + D
for i, panel in enumerate(panels):
    panel.vt = vt[i]
    
for panel in panels:
    panel.cp = 1.0 - (panel.vt)**2

In [ ]:
# Analytical solution
cp_exact = 1.0 - 4*(y_cylinder/R)**2

# 2. Flow around a NACA 0012 airfoil and comparing with theoritical solution. 

In [ ]:
#transforming into Joukowsky 
N_panels = 20
T  = 0.12
c = 1 
H = 0
a = c/4
theta = numpy.linspace(0, 2*math.pi, N_panels+1) 
Zeta_c = -4*a/(3*math.sqrt(3))*T+1j*2*H
r_s = ((1/4)+(T/(3*math.sqrt(3))))*c
 
Zeta = (r_s)*numpy.exp(1j*(theta))

J = (Zeta+Zeta_c) + a**2/(Zeta+Zeta_c)

x_ends = J.real
y_ends = J.imag

#x_ends = x_ends[::-1]
#y_ends = y_ends[::-1]

panels = numpy.empty(N_panels, dtype=object)
for i in range(N_panels):
    panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])
    

In [ ]:
A = numpy.empty((N_panels, N_panels), dtype='float')
#numpy.fill_diagonal(A, 0.5)
    
for i in range(N_panels):
    for j in range(N_panels):
        if i != j:
            A[i,j] = numpy.dot(get_source_panel_velocity(panels[j],panels[i].xc,panels[i].yc),panels[i].n)
        else:
            A[i,j] = 0.5
            
b = numpy.empty((N_panels), dtype='float')
for i in range(N_panels):
    b[i] = -freestream.u_inf.dot(numpy.array([u_inf,0]), (panels[i].n))


In [ ]:
sigma = numpy.linalg.solve(A,b)

In [ ]:
C = numpy.empty((N_panels, N_panels), dtype=float)

for i in range(N_panels):
    for j in range(N_panels):
        if i != j:
            C[i,j] = numpy.dot(get_source_panel_velocity(panels[j],panels[i].xc,panels[i].yc),panels[i].t)
        else:
            C[i,j] = 0.0
            
D = numpy.empty((N_panels), dtype='float')
for i in range(N_panels):
    D[i] = freestream.u_inf.dot(numpy.array([u_inf,0]), (panels[i].t))

In [ ]:
vt = numpy.dot(C,sigma) + D
for i, panel in enumerate(panels):
    panel.vt = vt[i]    
for panel in panels:
    panel.cp = 1.0 - (panel.vt)**2

In [ ]:
#Theoritical Solution.
voverVsquared=numpy.array([0, 0.64, 1.01, 1.241, 1.378, 1.402, 1.411, 1.411, 1.399, 1.378, 1.35, 1.288, 1.228, 1.166, 1.109, 1.044, 0.956, 0.906])
xtheo=numpy.array([0, 0.5, 1.25, 2.5, 5.0, 7.5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 95])
xtheo = xtheo/100
xtheo -=0.5

In [ ]:
# #transforming into Joukowsky 
pyplot.figure(figsize=(17,5))
pyplot.subplot(121)
pyplot.grid(True)
pyplot.xlabel('x', fontsize=16)
pyplot.ylabel('y', fontsize=16)

pyplot.plot(x_ends, y_ends, color='#CD2305', linestyle='-', linewidth=2)
pyplot.scatter([p.xa for p in panels], [p.ya for p in panels], color='#CD2305', s=10)
pyplot.scatter([p.xc for p in panels], [p.yc for p in panels], color='k', s=10, zorder=3)
pyplot.axis("equal")
pyplot.quiver([p.xc for p in panels], [p.yc for p in panels], [p.n[0] for p in panels] ,[p.n[1] for p in panels], scale = 10, color='k',zorder=3)

pyplot.legend([ 'panels', 'end-points', 'center-points'], 
           loc='best', prop={'size':16})

pyplot.subplot(122)
pyplot.grid(True)
pyplot.xlabel('x', fontsize=16);
pyplot.ylabel(r'$ C_p$', fontsize=16);
pyplot.title('Number of panels : %d' % N_panels);
pyplot.scatter([p.xc for p in panels], [p.cp for p in panels],color='#CD2305', s=40, zorder=2)
pyplot.plot(xtheo,1-voverVsquared, color='b',linewidth=2)
pyplot.xlim(-.6, .6)
pyplot.ylim(1.4, -.8);
pyplot.legend(['Analytical','Source panel Method'])

# 3. Sum of the flux rates due to all panels

In [ ]:
print("Flux Rate =", round(sum((sigma*[p.length for p in panels])),1)) 

Net flux rate has to be zero, as there is no flow entering or leaving the Airfoil
